## Smoothing test. Source https://notebook.community/naru-T/2017FOSS4GHOKKAIDO/FOSS4G_Hokkaido

In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()
import time
import datetime
import seaborn as sns

In [2]:
Map = geemap.Map()

### This exports filtered images for one small region


In [3]:
dataset = ee.ImageCollection('MODIS/006/MOD09A1').filter(ee.Filter.date('2002-01-01', '2002-12-31'))

In [4]:
maskMarco = ee.Image("users/marcogirardello/phenoutils/mask_unchanged_500m")

In [5]:
#----- Required functions
# add NDVI to data
def addNDVI(image):
    #image = image.updateMask(MakMarco.eq(1))
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('StateQA')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask) 

def smooth_func(image): 
    collection = ee.ImageCollection.fromImages(image.get('images'))
    return ee.Image(image).addBands(collection.mean().rename(['smooth']))

def setone(image):
    tmp = image.select('smooth').divide(image.select('smooth'))
    return tmp.toByte()

In [6]:
#run computations
# mask pixels on the basis of quality flags
MODmasked = dataset.map(maskPixels)
# add NDVI band
MODmaskedndvi = MODmasked.map(addNDVI).select('NDVI').filterDate('2002-01-01', '2002-12-31')

In [7]:
## This field contains UNIX time in milliseconds.
timeField = 'system:time_start'
join = ee.Join.saveAll('images')

interval = 32

##ee.Filter.maxDifference:  
##Creates a unary or binary filter that passes if the left and right operands, both numbers, are within a given maximum difference. If used as a join condition, this numeric difference is used as a join measure.
diffFilter =  ee.Filter.maxDifference(difference = (1000 * 60 * 60 * 24) * interval,
                                      leftField = timeField, 
                                      rightField = timeField)

NeighborJoin = join.apply(primary = MODmaskedndvi, 
                               secondary = MODmaskedndvi, 
                               condition = diffFilter)


smoothed = ee.ImageCollection(NeighborJoin.map(smooth_func))

In [8]:
smoothedint = smoothed.map(setone)
count = smoothedint.reduce(ee.Reducer.sum())
count1 = count.updateMask(maskMarco.eq(1))
complete = count1.updateMask(count1.eq(46))

In [9]:
# generate palette
pal = sns.color_palette("viridis",20).as_hex()
palette = {'min':1 , 'max':46, 'palette':pal}

In [10]:
Map.addLayer(count1,palette,'count forest')
Map.addLayer(complete,'','complete count')

In [11]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

### Export images (just a test)

In [ ]:
for image in images:
    print(image)
    # get image from list
    img1 = ee.Image(ndvilist.get(image))
    # create filename
    date = img1.get('system:time_start').getInfo()
    filename = 'file_'+str(date)
    # export tile
    task = ee.batch.Export.image.toDrive(image = img1,folder = 'phenology_8day',description=filename,
                                     scale=463.3127165275,crs = 'EPSG:4326',maxPixels = 1e13,region = aoi,
                                    skipEmptyTiles=True)
    task.start()

In [38]:
fimg = MODmaskedndvi.first()

In [40]:
# export tile
task = ee.batch.Export.image.toDrive(image = fimg,folder = 'phenology_8day',description='second_image',
                                     scale=463.3127165275,crs = 'EPSG:4326',maxPixels = 1e13,region = aoi,
                                    skipEmptyTiles=True)
task.start()